## Dealing with large numbers in Python

Today we're gonna be looking at a cryptography problem which we're going to be using as an excuse to learn how to deal with super large numbers in Python using only standard libraries, in a "do it yourself from scratch" manner :)

The problem we're gonna be looking at is "Sum-O-Primes" from the Cryptography category on PicoCTF, a site with cybersecurity contests(CTF) training problems. Please note that I did not make this problem, and all credits go to the respective creators. This problem comes from a past CTF event, so it's ok to share a solution and use the text we'll be decrypting on the site for some internet points if you want. You can check out the original problem at https://play.picoctf.org/practice/challenge/310?category=2&page=5

We are provided with 2 files to start, the implementation of the encryption algorithm as a .py file where we can read the code (bad_rsa.py in the files folder of the zip you found this notebook in), and a text file with the ciphertext (encrypted text we're supposed to decrypt) and 2 other numbers (out.txt in the files folder).
Let's read in the contents of this output file and store them as variables so we can work with them.

In [1]:
with open("files/out.txt", "r") as file:
    variables = [var[:-1] for var in file.readlines()] #we remove the last character from each line because it's the newline
    #character (\n)
    
print(variables)
#we skip the first 4 characters for each line to only store the actual value of each variable without the "x = " or similar
x = variables[0][4:]
n = variables[1][4:]
c = variables[2][4:]

['x = 1429cf99b5dd5dde9f016095be650d5b0a9a73e648aa72324cb8eb05bd14c1b913539a97f5417474f6014de830ad6dee028dd8908e593b1e99c4cc88f400127214036e71112292e58a2ccffc48f57524aee90f9858d935c7a86297a90c7fe48b80f6c4e8df9eaae501ef40da7984502457255fbc8a9e1574ec6ba210be134f192', 'n = 64fc90f5ca6db24f7bfc6419de407596d29a9ecda526101b8d0eff2181e9b8ed1538a1cbabe4dfc5bcd899976e7739f8b448815b50db36a994c5b1df97981d562c663113fc5ee84f3206aecd18248fb4e9bddf205c8119e8437f7d6522e63d05bc357ae4969a4b3000b8226f8d142c23c4e38cdb0c385bf9564e8a115e4c52b7a2e3a9073a5d99d7bec3bca6452cf0c1b8d8b6b123cc6a6980cf14088d6a2bbb5ed36b85cb0003e535bd16d79ad54ff5b26e62f57de074654493d3a26a149786d5fbf61b42c9305092eb018aa3db3cb18b24f188ae520bd18acf9ffced09a2ba302a520f6e2bfd8eea9adc01eb8ee941181694a3ab493e1aa53fbbbf2851a591', 'c = 56ed81bbc149701110f0a15e2e6078ab926d74ee2c11b804ae4fad4333a25c247f38bb74867922438d10ce529b75f5ee5e29ce71d6f704cc0644f7e78d60a2af8921fbc49326280e3f0c00f2769e837363cbb05dc3f30bda8fdc94111fb025008eae562ae57029d5c

#### But what are x, n and c? And why are there letters in what I said will be numbers?
Let's take a look at the source code to find out!

```
p = get_prime(1024)
q = get_prime(1024)

x = p + q
n = p * q

e = 65537

m = math.lcm(p - 1, q - 1)
d = pow(e, -1, m)

c = pow(FLAG, e, n)

print(f'x = {x:x}')
print(f'n = {n:x}')
print(f'c = {c:x}')
```

We can see from the code that:
- **x** is the **sum of p and q**
- **n** is the **product of p and q**
- **p and q** are 2 **very large randomly generated prime numbers**
- **c is the ciphertext** we should decipher

The reasoning for the strange letters in our numbers comes from those prints. A quick Google search on python :x formatting reveals that those numbers were printed out **in hex format**. That means that the numbers are in base 16 rather than the usual base 10.<br>
Instead of the usual 0-9 digits, hex numbers have digits in the range 0-f (where the order is 0...9 then a...f)</br>
Thankfully, we can easily convert hex (base 16) into decimal (base 10) using the built-in **int** function by specifying the base.<br>
Let's take a look at some examples below:</br>

In [2]:
hex_nums = ["91", "abc", "ff", "2e", "11", "a0"]

for num in hex_nums:
    print(f"The hex number {num} corresponds to the decimal number {int(num,16)}")

The hex number 91 corresponds to the decimal number 145
The hex number abc corresponds to the decimal number 2748
The hex number ff corresponds to the decimal number 255
The hex number 2e corresponds to the decimal number 46
The hex number 11 corresponds to the decimal number 17
The hex number a0 corresponds to the decimal number 160


**But why is this the case?**<br>
Let's take the first example, 91.<br>
In decimal (base 10), the value of 91 is equivalent to:<br>
$9 \cdot 10^1 + 1 \cdot 10^0 = 9 \cdot 10 + 1 \cdot 1 = 90 + 1 = 91$<br>
In hex (base 16), the value of 91 is equivalent to:<br>
$9 \cdot 16^1 + 1 \cdot 16^0 = 9 \cdot 16 + 1 \cdot 1 = 144 + 1 = 145$<br>

Let's do the next one as well, abc:<br>
"abc" doesn't make make sense as a decimal (base 10) number, as the digits in base 10 range between 0-9.<br>
It does, however, make sense in hex (base 16), where the digits are in the range 0-f.<br>
Note that the order of the digits is 0 1 2 3 4 5 6 7 8 9 a b c d e f, so "a" coresponds to 10, "b" to 11, ..., "f" to 15<br>
$a \cdot 16^2 + b \cdot 16^1 + c \cdot 16^0 = 10 \cdot 256 + 11 \cdot 16 + 12 \cdot 1 = 2560 + 176 + 12 = 2748$

This generalizes for any number of digits. You can find a good video tutorial on how to convert a hex number to a decimal number here: https://www.youtube.com/watch?app=desktop&v=pg-HEGBpCQk

**Quick practice exercise**<br>
Convert the following numbers from hex to decimal:<br>
- 5
- 3d
- 20a1

**Answers**
<details>
5, 61, 8353<br>
Detailed calculations<br>
5 to decimal:<br>
$5 \cdot 16^0 = 5 \cdot 1 = 5$<br>
3d to decimal:<br>
$3 \cdot 16^1 + d \cdot 16^0 = 3 \cdot 16 + 13 \cdot 1 = 48 + 13 = 61$<br>
20a1 to decimal:<br>
$2 \cdot 16^3 + 0 \cdot 16^2 + a \cdot 16^1 + 1 \cdot 16^0 = 2 \cdot 4096 + 0 + 10 \cdot 16 + 1 \cdot 1 = 8192 + 160 + 1 = 8353$<br>

Now that we know how to convert hex numbers to decimal both manually and using the int() function, let's convert our given hex numbers, x, n and c to decimal numbers.

In [3]:
x = int(x, 16)
n = int(n, 16)
c = int(c, 16)

x, n, c

(226546681856190263619892616792218130521570032262900236549842672089682515778208912828199358515201866805387062463737665662193399729692506388610482184894766897264590903874257311731561568385258790764817634614996008408916675362993565156633617744400865408980221036805932997456044401299988657076337415409658280341906,
 12748375556570078931323876348697038029886491950060893186497892699683317310111501916404065392536144464445274333083913910180253319432381125194397252750773181597311793438448103802013908804250712055956932783382160826057817857884706400126928785524023416639273133188054126549943840133256364632699489374831153112912849257351235687536993709519898200527506046169055543255426445369812520133191225616000823928660305132530452791031554731459373681724346453451955400046033943647503543004239988261829585365322656463554537121162528508154784790226768384873003984706798967595704521420442755894365726841231719003175270545612919345882513,
 1097361318102537383649430345091713098705966183673445834521570685186

Damn! Those are truly some big numbers!<br>
Now that we got that whole hex problem out of the way, how do we actually decrypt c?<br>
We are told in the text of the original problem that the secret message is encrypted with an algorithm called RSA.<br>
A quick Google search on "RSA decryption formula" shows us that this is the formula to decrypt a message that was encrypted with RSA:<br>
$plaintext = C^d(mod \ n)$<br>
This seems complicated, but translated into python it would be just a simple pow(c, d, n), if we had c, d and n. <br>
Here, c is our ciphertext, we already have that in our "c" variable, and n is just the product of the two primes, which we also have in our "n" variable. But we don't seem to have a "d", what is that? <br>

Let's take a look at the code again, this time to see how we can get that "d" we need<br>
```
x = p + q
n = p * q

e = 65537

m = math.lcm(p - 1, q - 1)
d = pow(e, -1, m)

c = pow(FLAG, e, n)
```

The "d" in the code is the d we need. So how can we get that? <br>
```
d = pow(e, -1, m)
```
We can just copy and run the same code they wrote to get d, but we will need all of the variables initialized with the right values. We can see from the code that e = 65537, so we have that, but we don't have a value for "m". So let's go a step back and look at how they calculated m. <br>
```
m = math.lcm(p - 1, q - 1)
```
"math.lcm" just calculates the lowest common multiple, and again, once we have all of the variables we can just copy paste this line and run it to get m, so we don't need to worry about how this function works under the hood. However, it seems that we need p and q, the 2 very large randomly generated primes. <br>
If we try to look back once more and find some clues on how to calculate p and q, we will just come across how they were initialized <br>
```
p = get_prime(1024)
q = get_prime(1024)
```
p and q are, indeed, 2 randomly generated very large primes, and there is nothing we can do with this information alone to find out what their values are.<br>

**So what did we find out?**<br>
We need to somehow find p and q, the 2 randomly generated very large primes. If we succeed that, we can get the other necessary variables m and d, and decrypt the message by copy pasting code that does some simple math.<br>
**Now, how can we actually do that?**<br>
Well, the simple answer is, in proper implementations of RSA, you can't.<br>
Usually, you would only get N, the public key, the product of the 2 primes, p and q, so if you wanted to decrypt the message without knowing d, the private key, you would need to somehow get p and q solely from their product N. You don't need to worry about what the terms _public key_ and _private key_ mean and the theory behind how this works, but, as you probably guessed, you need both of those to decrypt the message.<br>
Given that p and q are large enough and all of the other variables are set to appropriate values when encrypting the message, it is **computationally infeaseable** (_fancy terminology_ for "would take forever and it's not doable") to find p and q and decrypt the message. The whole concept of RSA, which is one of the most commonly used encryption algorithms, which is considered secure, is that it's very hard to find the 2 primes from their product.<br>

Now, you might be thinking _Amazing, well how is this problem even solvable then?_<br>
Well, since this problem is posted on a _training site_ you probably guessed that it was made specifically to be solved, and this implementation must be somehow flawed to allow you to actually decrypt the message. <br>

Here, the key lies in the **_extra information_** we are given: x<br>
In this problem, x is the sum of the 2 primes p and q, and it's extra information that is not usually (and should'nt be, as you will soon see) given.<br>
With the given information, we now have 2 different pieces of information about p and q.<br>
- p + q = x
- p * q = n<br>

Where we are given x and n:

In [4]:
print(f"The sum of p and q (x) is {x}\n")
print(f"The product of p and q (n) is {n}")

The sum of p and q (x) is 226546681856190263619892616792218130521570032262900236549842672089682515778208912828199358515201866805387062463737665662193399729692506388610482184894766897264590903874257311731561568385258790764817634614996008408916675362993565156633617744400865408980221036805932997456044401299988657076337415409658280341906

The product of p and q (n) is 12748375556570078931323876348697038029886491950060893186497892699683317310111501916404065392536144464445274333083913910180253319432381125194397252750773181597311793438448103802013908804250712055956932783382160826057817857884706400126928785524023416639273133188054126549943840133256364632699489374831153112912849257351235687536993709519898200527506046169055543255426445369812520133191225616000823928660305132530452791031554731459373681724346453451955400046033943647503543004239988261829585365322656463554537121162528508154784790226768384873003984706798967595704521420442755894365726841231719003175270545612919345882513


That surely does look intimidating.<br>
However, there is actually an extremely simple way to find **any 2 numbers if you are given their sum and product**<br>

Let's take a much simpler example:
- the sum of the numbers is 9
- the product of the numbers is 18<br>

What are the numbers?<br>
**Answer**
<details>3 and 6

That was probably not too hard to think of, but what if we had bigger numbers, like the sum being 108 and the product being 2835?<br>
Probably still very guessable, but considerably harder. Guessing for those huge numbers from above is certainly out of the question.<br>

We can actually automatise this little guessing game to work for any sum and product using the properties of the 2nd degree equation.<br>
You may remember that there's actually a theorem regarding the sum and product of 2nd degree equation roots.<br>
It's called **Vieta's formula** and you've probably seen it at some point in high school:<br>

### Vieta's formula

For a 2nd degree equation $ax^2 + bx + c$ with roots $x_1$ and $x_2$<br>
$S = x_1 + x_2 = \frac{-b}{a}$<br>
$P = x_1 \cdot x_2 = \frac{c}{a}$<br>

But what if we set a to 1?<br>
Then S would be -b, and P would be c.<br>
Which would mean that we can rewrite the 2nd degree equation like this:<br>
$x^2 - Sx + P$<br><br>
What would we get if we solved this simple 2nd degree equation? $x_1$ and $x_2$, the numbers that were used to get S and P, the sum and product that we were given.<br>
**This means that, in theory, we should be able to get any 2 numbers given their sum and product by solving a simple 2nd degree equation**<br>
If you're interested, you can read more about Vieta's formula here: https://www.geeksforgeeks.org/vietas-formula/

Let's attempt exactly that for our guessing game!<br>
All we need is to write a function to solve a 2nd degree equation.<br>
We will just use the good old math formulas:<br>
$\Delta = b^2 - 4ac$<br>
$x_1 = \frac{-b - \sqrt{\Delta}}{2a}$<br>
$x_2 = \frac{-b + \sqrt{\Delta}}{2a}$<br>

In [5]:
def solve_2nd_degree_eq(a,b,c):
    
    delta = (b ** 2) - (4 * a * c)
    
    x_1 = (-b - (delta ** (1/2))) / (2 * a)
    x_2 = (-b + (delta ** (1/2))) / (2 * a)
    
    return x_1, x_2

Now all we need to do is to pass the 2nd degree equation function a=1, b=S and c=P.<br>
Let's try that for our examples from before:<br>

In [6]:
S = 9
P = 18
x1, x2 = solve_2nd_degree_eq(1,-S,P)
print(f"Calculated x1, x2: {x1} and {x2}")
print("Checks:")
print(f"Sum check: {x1 + x2 == S}")
print(f"Product check: {x1 * x2 == P}")

Calculated x1, x2: 3.0 and 6.0
Checks:
Sum check: True
Product check: True


In [7]:
S = 108
P = 2835
x1, x2 = solve_2nd_degree_eq(1,-S,P)
print(f"Calculated x1, x2: {x1} and {x2}")
print("Checks:")
print(f"Sum check: {x1 + x2 == S}")
print(f"Product check: {x1 * x2 == P}")

Calculated x1, x2: 45.0 and 63.0
Checks:
Sum check: True
Product check: True


This seems to be working amazingly! Let's try some much bigger numbers!

In [8]:
secret_a = 79282929
secret_b = 29202929

In [9]:
S = secret_a + secret_b
P = secret_a * secret_b
x1, x2 = solve_2nd_degree_eq(1,-S,P)
print(f"Calculated x1, x2: {x1} and {x2}")
print("Checks:")
print(f"Sum check: {x1 + x2 == S}")
print(f"Product check: {x1 * x2 == P}")

Calculated x1, x2: 29202929.0 and 79282929.0
Checks:
Sum check: True
Product check: True


It seems that it still works! But what is we go **extra extra large numbers**??

In [10]:
secret_a = 37289292200273738383392929292902027
secret_b = 38393839839226622728722782728282787

In [11]:
S = secret_a + secret_b
P = secret_a * secret_b
x1, x2 = solve_2nd_degree_eq(1,-S,P)
print(f"Calculated x1, x2: {x1} and {x2}")
print("Checks:")
print(f"Sum check: {x1 + x2 == S}")
print(f"Product check: {x1 * x2 == P}")

Calculated x1, x2: 3.728929220027374e+34 and 3.8393839839226623e+34
Checks:
Sum check: False
Product check: False


Oh no! It seems that this implementation doesn't pass the _keyboard smash tier large numbers_ check<br>
Even worse, if we try to run this with the numbers from the problem we're actually supposed to use (x,n) it won't even run.<br>

In [12]:
#The line below generates an error, uncomment to see
#solve_2nd_degree_eq(1,-x,n)

What is this cryptic _OverflowError: int too large to convert to float_?<br>
Well, python can deal with arbitrarily large ints.

In [13]:
2 ** 2822

3211055143340940382464701995841782424686400638977648259632150521461986613550578643864445468841016012001672174511554692941376598035060853331634624453063827342736461010075969149908416669710297818346625849032331322187398357787767922588780553070264826136305061951454770208747429689676454931530240791609435716680897597090263587597011947663894280468370901108568527442613474033782355564354626660297569063324026070521616741278315837078139140574279874184337544622269075058589143864925039317811327098188921655020581719341347320576052208984680999675025116359553289010546629423232177330396979855667584182050003018218664998361488247466863011537015930399172338480034352541520994486195402495359225929187167485155790019006222010407617918119086357727106822432233666397804275022205465023644431616429078584885544585631277677667098388784722017625861313517682100277346304

That was completely fine! What a number!<br>
But python is definitely not as generous in the float department

In [14]:
#The line below generates an error, uncomment to see
#(2 ** 2822) ** (1/2)

It seems that trying to take the square root of our super large resulting delta generates the _OverflowError: int too large to convert to float_ error.<br>
Unfortunately, we'd be facing a similar problem with the following division as well.

In [15]:
#The line below generates an error, uncomment to see
#(2 ** 2822) / 1000

We now get an _OverflowError: integer division result too large for a float_.<br>
Well, we have 2 different errors, and even if the code went through the result wouldn't be accurate.<br>
That doesn't seem great.

It seems (upon a few intense Google searches) that all those 3 problems happen because of the same thing: python's limited float precision.<br>
Fortunately, there is a specialized library in standard python that deals with exactly these types of problems: **decimal**.<br>
You can read more about it in the official python documentation: https://docs.python.org/3/library/decimal.html

In [16]:
from decimal import *

This library allows us to use the data type _Decimal_, which is based on _float_, but allows for setting precision as big as needed (unlike the classical float), allowing us to deal with those super large floats we encountered.<br>
By default, the precision will be 28 digits after the floating point, we can see this with getcontext().<br>
We can change this precision with getcontext().prec<br>

In [17]:
#default context
getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [18]:
getcontext().prec = 1000
getcontext()

Context(prec=1000, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

We will be using Decimal.sqrt() instead of ** 1/2 to make use of the arbitrary precision of the Decimal module (and because the built in power function (** and pow) doesn't work with the _Decimal_ data type).

In [19]:
Decimal(2 ** 2822).sqrt()

Decimal('56666172831248630060015497598719762697896336582016192354183482017064171595901376302024618239812323953109806145875966553841429596923860187556201738611615082360670359226839961894879598513270888850718347987420163226284508037579500049230358987876492590073311013873374604982140967663399026943753480074867073035393892114351651662868044658104820561250062374112394894832424918954576789805774061036915911725078996135856097829792514048')

Now we can deal with square roots of arbitrarily large numbers! Hooray!

In [20]:
Decimal(2 ** 2822) / Decimal(1000)

Decimal('3211055143340940382464701995841782424686400638977648259632150521461986613550578643864445468841016012001672174511554692941376598035060853331634624453063827342736461010075969149908416669710297818346625849032331322187398357787767922588780553070264826136305061951454770208747429689676454931530240791609435716680897597090263587597011947663894280468370901108568527442613474033782355564354626660297569063324026070521616741278315837078139140574279874184337544622269075058589143864925039317811327098188921655020581719341347320576052208984680999675025116359553289010546629423232177330396979855667584182050003018218664998361488247466863011537015930399172338480034352541520994486195402495359225929187167485155790019006222010407617918119086357727106822432233666397804275022205465023644431616429078584885544585631277677667098388784722017625861313517682100277346.304')

And we can also divide arbitrarily large numbers!<br>
Let's upgrade our function to work for super big numbers now :)

In [21]:
def solve_2nd_degree_eq_fancy(a,b,c):
    
    delta = (b ** 2) - (4 * a * c)
    
    sqrt_delta = Decimal(delta).sqrt()
    x_1 = (Decimal(-b) - sqrt_delta) / Decimal(2 * a)
    x_2 = (Decimal(-b) + sqrt_delta) / Decimal(2 * a)
    
    return int(x_1), int(x_2)

Let's test it out on our super big number test:

In [22]:
secret_a = 37289292200273738383392929292902027
secret_b = 38393839839226622728722782728282787
S = secret_a + secret_b
P = secret_a * secret_b
x1, x2 = solve_2nd_degree_eq_fancy(1,-S,P)
print(f"Calculated x1, x2: {x1} and {x2}")
print("Checks:")
print(f"Sum check: {x1 + x2 == S}")
print(f"Product check: {x1 * x2 == P}")

Calculated x1, x2: 37289292200273738383392929292902027 and 38393839839226622728722782728282787
Checks:
Sum check: True
Product check: True


This seems to work now.<br>
It's time for the real test, running it on our given numbers, x and n.

In [23]:
p, q = solve_2nd_degree_eq_fancy(1,-x,n)
print(f"Calculated x1, x2: {p} and {q}")
print("Checks:")
print(f"Sum check: {p + q == x}")
print(f"Product check: {p * q == n}")

Calculated x1, x2: 104191809755303135841130553919690511982590784006703342137891233625303583575261037741899643858912976657685104564581344274225446213011385644579286861030288322940001431308705375156622789940928620617537019250142554607990319581184398156171545104914273931427544198611355393768325227410092849953567948286813698778689 and 122354872100887127778762062872527618538979248256196894411951438464378932202947875086299714656288890147701957899156321387967953516681120744031195323864478574324589472565551936574938778444330170147280615364853453800926355781809167000462072639486591477552676838194577603687719173889895807122769467122844581563217
Checks:
Sum check: True
Product check: True


We have now gotten the code to run with no errors, as well as return p and q which do actually give the expected sum and product.<br>
Now all we have left to do is to copy paste the code they provided with our variables to get m and d, then apply the formula to decrypt our ciphertext c.<br>

In [24]:
import math

In [25]:
e = 65537

m = math.lcm(p - 1, q - 1)
d = pow(e, -1, m)

In [26]:
plaintext = pow(c,d,n)
plaintext

19023464378915228186961971911578673930414528675538075162786456500620236949484333128408678850102656856786656603493716332838663549

Is this correct? What is this number?<br>
If we look at the source code, we can see that the secret message (the "flag") was first encoded as hex, and then converted to an int, so let's try to reverse that process.<br>
```
FLAG  = int(hexlify(FLAG.encode()), 16)
```
Originally, we know we had some english plain text, then, it was converted to hex, and this hex was then converted to int.<br>
Let's do that process in reverse:<br>
int -> hex -> string (ASCII)

Convert int to hex using built in hex() function

In [27]:
plaintext = hex(plaintext)
plaintext

'0x7069636f4354467b706c33337a5f6e305f673176335f63306e677275336e63335f30665f357175347233355f32343932396334357d'

We will need to get rid of the "0x" before the number, otherwise the next function we'll use will throw an error

In [28]:
plaintext[2:]

'7069636f4354467b706c33337a5f6e305f673176335f63306e677275336e63335f30665f357175347233355f32343932396334357d'

In [29]:
#convert hex string to bytes so we can use the built in decode method
bytes_obj = bytes.fromhex(plaintext[2:])
result_string = bytes_obj.decode('ascii')
print(result_string)

picoCTF{pl33z_n0_g1v3_c0ngru3nc3_0f_5qu4r35_24929c45}


We did it!<br>
We deciphered the secret message (the "flag") and got 400 juicy internet points using our mad math skills!<br>
On this journey we learned:<br>
- What hex is, how to convert from hex to decimal both manually and using the int(hexstr, 16) function, as well as how to convert from decimal to hex using the hex() function
- Vieta's formula and how to use it
- The limits of the default float type
- How to deal with arbitrarily large numbers using the decimal module

P.S: There might be faster or better ways to solve this problem. What we did in this notebook is actually much more general than it was needed for the problem (our implementation works for any 2 ints, and we only needed it to work for primes of a certain size).<br>
Maybe some automatic tools would even solve this for you, but regardless, I hope you found this interesting and you enjoyed the process :)